In [1]:
import duckdb
import pandas as pd

# Connect to your dbt DuckDB file
con = duckdb.connect("../stock_dbt_project/dev.duckdb")



con.execute("SHOW TABLES").fetchall()


[('features_stock_data',),
 ('my_first_dbt_model',),
 ('my_second_dbt_model',),
 ('stg_stock_data',)]

In [2]:
df = con.execute("SELECT * FROM features_stock_data").fetchdf()

In [3]:
df['datetime'] = pd.to_datetime(df['datetime'])

# Sort for time series modeling
df = df.sort_values(by=['symbol', 'datetime'])

# Preview
df.head()


,symbol,datetime,open,high,low,close,volume,prev_close,rolling_avg_5,rolling_std_5,daily_range,avg_volume_5,daily_return,volume_ratio,rolling_volatility_10,sector
26487,AAPL,2025-01-13 14:30:00,233.500000,233.589996,229.720001,230.380005,13314474,230.380005,230.380005,0.000000,3.869995,1.331447e+07,0.000000,1.000000,NaN,Technology
26488,AAPL,2025-01-13 14:30:00,233.500000,233.589996,229.720001,230.380005,13314474,NaN,230.380005,NaN,3.869995,1.331447e+07,NaN,1.000000,NaN,Technology
26489,AAPL,2025-01-13 15:30:00,230.380005,231.800003,230.220001,230.725006,5357189,230.380005,230.495005,0.199187,1.580002,1.066205e+07,0.149753,0.502454,0.105891,Technology
26490,AAPL,2025-01-13 15:30:00,230.380005,231.800003,230.220001,230.725006,5357189,230.725006,230.552505,0.199187,1.580002,9.335832e+06,0.000000,0.573831,0.086460,Technology
26491,AAPL,2025-01-13 16:30:00,230.732498,232.529999,230.399994,232.320007,4160495,230.725006,230.906006,0.809054,2.130005,8.300764e+06,0.691300,0.501218,0.328369,Technology


In [ ]:
df.shape

In [4]:
df['symbol'].value_counts()



symbol
AAPL    1537
AXP     1349
CRM     1196
CAT     1196
DIS     1196
DOW     1196
CSCO    1196
CVX     1196
GS      1196
HD      1196
IBM     1196
HON     1196
MRK     1196
NKE     1196
INTC    1196
JNJ     1196
JPM     1196
KO      1196
MCD     1196
MMM     1196
MSFT    1110
AMGN     920
BA       882
PG       622
TRV      314
V        314
VZ       314
WBA       81
Name: count, dtype: int64

In [5]:

symbol_counts = df['symbol'].value_counts()
valid_symbols = symbol_counts[symbol_counts > 300].index.tolist()

df = df[df['symbol'].isin(valid_symbols)]

print("Symbols retained:", valid_symbols)


Symbols retained: ['AAPL', 'AXP', 'CRM', 'CAT', 'DIS', 'DOW', 'CSCO', 'CVX', 'GS', 'HD', 'IBM', 'HON', 'MRK', 'NKE', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MSFT', 'AMGN', 'BA', 'PG', 'TRV', 'V', 'VZ']


In [6]:
features = [
    'open', 'high', 'low', 'volume',
    'rolling_avg_5', 'rolling_std_5',
    'daily_range', 'avg_volume_5',
    'daily_return', 'volume_ratio', 'rolling_volatility_10'
]

In [7]:
X = df[features]
y = df['close']

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [10]:
from dotenv import load_dotenv
import os



In [11]:
TRACKING_SERVER_HOST = os.getenv('MLFLOW_TRACKING_URI')
print(TRACKING_SERVER_HOST)

http://ec2-3-89-152-136.compute-1.amazonaws.com:5000


In [12]:
import mlflow
mlflow.set_tracking_uri(f"{TRACKING_SERVER_HOST}")


In [13]:
print(mlflow.get_tracking_uri())

http://ec2-3-89-152-136.compute-1.amazonaws.com:5000


In [14]:
import boto3

# Initialize a session, optionally passing the region if necessary
session = boto3.Session(region_name='us-east-1')

# Get the credentials from the session
credentials = session.get_credentials().get_frozen_credentials()

# Get the region from the session
region = session.region_name

# Print credentials and region
# print(f"AWS Access Key ID: {credentials.access_key}")
# print(f"AWS Secret Access Key: {credentials.secret_key}")
# print(f"AWS Region: {region}")

s3_client = session.client('s3')

In [15]:
mlflow.set_experiment("Stock_Price_Prediction")

<Experiment: artifact_location='s3://mlflow-artifacts-remote23/249678797774142465', creation_time=1746053039574, experiment_id='249678797774142465', last_update_time=1746053039574, lifecycle_stage='active', name='Stock_Price_Prediction', tags={}>

In [16]:
import xgboost as xgb
import mlflow
import mlflow.xgboost

In [19]:
from mlflow.models.signature import infer_signature
from mlflow.tracking import MlflowClient

In [18]:
with mlflow.start_run(run_name="xgboost_stock_model"):
    model = xgb.XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=6,
        random_state=42,
        tree_method="hist"
    )

    model.fit(X_train, y_train)

    # Predict
    preds = model.predict(X_test)
    train_preds = model.predict(X_train)
    rmse = mean_squared_error(y_test, preds)
    signature = infer_signature(X_train, train_preds)
    print(f"Test RMSE: {rmse:.4f}")

    # Log parameters
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("learning_rate", 0.1)
    mlflow.log_param("max_depth", 6)

    # Log metric
    mlflow.log_metric("rmse", rmse)

    # Log model artifact
    mlflow.xgboost.log_model(
    model,
    artifact_path="models/xgboost_stock_model",
    signature=signature,
    input_example=X_train.iloc[:5],
    registered_model_name="StockPricePredictor"
)

print("✅ Model training + MLflow logging complete!")

c:\Users\loges\.virtualenvs\Stock_price-P2gocZed\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Test RMSE: 2.1061


c:\Users\loges\.virtualenvs\Stock_price-P2gocZed\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [21:55:59] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
INFO:botocore.credentials:Found credentials in environment variables.
Registered model 'StockPricePredictor' already exists. Creating a new version of this model...
2025/05/23 21:56:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: StockPricePredictor, version 2
Created version '2' of model 'StockPricePredictor'.


🏃 View run xgboost_stock_model at: http://ec2-3-89-152-136.compute-1.amazonaws.com:5000/#/experiments/249678797774142465/runs/9e6bd859a825413280349b498ef1f9f9
🧪 View experiment at: http://ec2-3-89-152-136.compute-1.amazonaws.com:5000/#/experiments/249678797774142465
✅ Model training + MLflow logging complete!


In [20]:


client = MlflowClient()

model_name = "StockPricePredictor"

# Get all versions of the registered model
all_versions = client.get_latest_versions(name=model_name, stages=["None", "Staging", "Archived"])

# Sort by version number (as int) to find the latest
latest_version = max(all_versions, key=lambda v: int(v.version))

print(f"Promoting version {latest_version.version} of model '{model_name}' to Production...")

# Promote to Production (archives any existing version in Production)
client.transition_model_version_stage(
    name=model_name,
    version=latest_version.version,
    stage="Production",
    archive_existing_versions=True
)

print(f"✅ Model version {latest_version.version} is now in Production.")


C:\Users\loges\AppData\Local\Temp\ipykernel_38140\3700713261.py:6: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  all_versions = client.get_latest_versions(name=model_name, stages=["None", "Staging", "Archived"])


Promoting version 2 of model 'StockPricePredictor' to Production...


C:\Users\loges\AppData\Local\Temp\ipykernel_38140\3700713261.py:14: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


✅ Model version 2 is now in Production.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Filter for AAPL
aapl_df = df[df['symbol'] == 'AAPL'].sort_values('datetime')

# Plot closing price
plt.figure(figsize=(12, 5))
sns.lineplot(x='datetime', y='close', data=aapl_df)
plt.title("AAPL Closing Price Over Time")
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
model_uri = "models:/StockPricePredictor/1"  # version 1 of registered model
model = mlflow.pyfunc.load_model(model_uri)

In [ ]:
import pandas as pd

sample_input = pd.DataFrame([{
    "open": 170.0,
    "high": 172.0,
    "low": 168.5,
    "volume": 15000000,
    "rolling_avg_5": 171.2,
    "rolling_std_5": 1.3,
    "daily_range": 3.5,
    "avg_volume_5": float(14500000),
    "daily_return": 0.5,
    "volume_ratio": 1.05,
    "rolling_volatility_10": 1.2,
    "sector_Consumer_Discretionary": 0,
    "sector_Consumer_Staples": 0,
    "sector_Energy": 0,
    "sector_Financials": 0,
    "sector_Healthcare": 0,
    "sector_Industrials": 0,
    "sector_Materials": 0,
    "sector_Communication_Services": 0,
    "sector_Technology": 1,  # assume AAPL
    "sector_Unknown": 0,
}])

# Predict
prediction = model.predict(sample_input)
print("✅ Prediction:", prediction[0])